In [74]:
import pandas as pd 
from neo4j import GraphDatabase

In [ ]:
## seif credentials
# uri = "neo4j+ssc://e45fa759.databases.neo4j.io"
# user = "neo4j"
# password = "ViO4jopeE4TjckK7VCChP6q6JOXn0DBVl5E8NhvMqv8"

#abushanab credentials
uri = "neo4j+ssc://b0d23506.databases.neo4j.io"
user = "neo4j"
password = "yA219NUjGwZeHjL7lQgGUHfzSJsQtneBlXqGByjik8c"

driver = GraphDatabase.driver(uri , auth=(user, password))

with driver.session() as session:
    print(session.run("RETURN 1").single())

<Record 1=1>


In [76]:
df = pd.read_csv("fpl_two_seasons.csv")

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51952 entries, 0 to 51951
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             51952 non-null  object 
 1   name               51952 non-null  object 
 2   position           51952 non-null  object 
 3   assists            51952 non-null  int64  
 4   bonus              51952 non-null  int64  
 5   bps                51952 non-null  int64  
 6   clean_sheets       51952 non-null  int64  
 7   creativity         51952 non-null  float64
 8   element            51952 non-null  int64  
 9   fixture            51952 non-null  int64  
 10  goals_conceded     51952 non-null  int64  
 11  goals_scored       51952 non-null  int64  
 12  ict_index          51952 non-null  float64
 13  influence          51952 non-null  float64
 14  kickoff_time       51952 non-null  object 
 15  minutes            51952 non-null  int64  
 16  own_goals          519

# Creating Nodes

## 1. Season: [season_name]

In [78]:
def createSeason(tx ,season_name):
    tx.run("CREATE (s:Season {season_name: $season_name})", season_name=season_name)

In [79]:
# .unique 3shan single value

unique_seasons = df['season'].dropna().unique()
unique_seasons

array(['2021-22', '2022-23'], dtype=object)

In [80]:
with driver.session() as session:
    for season in unique_seasons:
        session.execute_write(createSeason, season)  


## 2. Gameweek: [season, GW_number]


In [81]:
def createGW(tx, season , GW_number):
    tx.run("CREATE (gw:Gameweek {season: $season , GW_number: $GW_number})", season=season , GW_number=GW_number)

In [82]:
# .drop_duplicates not .unique 3shan da msh single value 

GWperSeason = df[['season', 'GW']].sort_values(by=['season', 'GW']).drop_duplicates()
GWperSeason

,season,GW
0,2021-22,1
1,2021-22,2
2,2021-22,3
3,2021-22,4
4,2021-22,5
...,...,...
108,2022-23,34
110,2022-23,35
111,2022-23,36
112,2022-23,37


## we realized that season 22-23 gw 7 is missing

In [83]:
for index, row in GWperSeason.iterrows():
    print(row['season'], row['GW'])


2021-22 1
2021-22 2
2021-22 3
2021-22 4
2021-22 5
2021-22 6
2021-22 7
2021-22 8
2021-22 9
2021-22 10
2021-22 11
2021-22 12
2021-22 13
2021-22 14
2021-22 15
2021-22 16
2021-22 17
2021-22 18
2021-22 19
2021-22 20
2021-22 21
2021-22 22
2021-22 23
2021-22 24
2021-22 25
2021-22 26
2021-22 27
2021-22 28
2021-22 29
2021-22 30
2021-22 31
2021-22 32
2021-22 33
2021-22 34
2021-22 35
2021-22 36
2021-22 37
2021-22 38
2022-23 1
2022-23 2
2022-23 3
2022-23 4
2022-23 5
2022-23 6
2022-23 8
2022-23 9
2022-23 10
2022-23 11
2022-23 12
2022-23 13
2022-23 14
2022-23 15
2022-23 16
2022-23 17
2022-23 18
2022-23 19
2022-23 20
2022-23 21
2022-23 22
2022-23 23
2022-23 24
2022-23 25
2022-23 26
2022-23 27
2022-23 28
2022-23 29
2022-23 30
2022-23 31
2022-23 32
2022-23 33
2022-23 34
2022-23 35
2022-23 36
2022-23 37
2022-23 38


In [84]:
with driver.session() as session:
    for _, row in GWperSeason.iterrows():
        session.execute_write(createGW, row["season"], row["GW"])


BufferError: Existing exports of data: object cannot be re-sized

## 3. Fixture: [season, fixture_number], kickoff _time

In [ ]:
def createFixture(tx, season, fixture_number, kickoff_time):
    tx.run(
        """
        MERGE (f:Fixture {season: $season, fixture_number: $fixture_number})
        SET f.kickoff_time = $kickoff_time
        """,
        season=season,
        fixture_number=fixture_number,
        kickoff_time=kickoff_time
    )

In [ ]:
Fixtures = df[['season','fixture','kickoff_time']].drop_duplicates(subset=['season','fixture'])
Fixtures


,season,fixture,kickoff_time
0,2021-22,2,2021-08-14 14:00:00+00:00
1,2021-22,12,2021-08-21 16:30:00+00:00
2,2021-22,22,2021-08-28 14:00:00+00:00
3,2021-22,32,2021-09-11 14:00:00+00:00
4,2021-22,42,2021-09-19 13:00:00+00:00
...,...,...,...
4671,2021-22,34,2021-09-11 11:30:00+00:00
4680,2021-22,140,2021-12-02 19:30:00+00:00
4683,2021-22,188,2021-12-26 15:00:00+00:00
6826,2021-22,13,2021-08-21 14:00:00+00:00


In [ ]:
with driver.session() as session:
    for _, row in Fixtures.iterrows():
        session.execute_write(createFixture, row["season"], row["fixture"] , row['kickoff_time'])

## 4. Team: [name]



In [ ]:
def createTeam(tx , name):
    tx.run("CREATE (t:Team {name: $name})", name=name)

In [ ]:
teamNames = df['home_team'].unique()
teamNames.size
teamNames

array(['Burnley', 'Brighton', 'Brentford', 'Crystal Palace', 'Norwich',
       'Liverpool', 'Aston Villa', 'West Ham', 'Southampton', 'Chelsea',
       'Everton', 'Leicester', 'Watford', 'Man Utd', 'Newcastle',
       'Arsenal', 'Spurs', 'Man City', 'Wolves', 'Leeds', "Nott'm Forest",
       'Fulham', 'Bournemouth'], dtype=object)

In [ ]:
with driver.session() as session:
    for team in teamNames:
        session.execute_write(createTeam, team)  

## 5.Player: [player_name, player_element]

In [ ]:
def createPlayer(tx , player_name , player_element):
    tx.run("CREATE (pn:Player {player_name: $player_name , player_element: $player_element})", player_name=player_name , player_element=player_element)

In [ ]:
players = df[['name' , 'element']].drop_duplicates()
players

,name,element
0,Aaron Connolly,72
38,Aaron Cresswell,411
76,Aaron Cresswell,457
114,Aaron Hickey,510
152,Aaron Lennon,575
...,...,...
51796,Zidane Iqbal,551
51834,Álvaro Fernández,556
51866,Álvaro Fernández,728
51876,Çaglar Söyüncü,217


In [ ]:
with driver.session() as session:
    for _, row in players.iterrows():
        session.execute_write(createPlayer, row["name"], row["element"])

## 6.Position: [name]

In [ ]:
def createPosition(tx , name):
    tx.run("CREATE (t:Position {name: $name})", name=name)

In [ ]:
positions = df['position'].unique()
positions

array(['FWD', 'DEF', 'MID', 'GK'], dtype=object)

In [ ]:
with driver.session() as session:
    for position in positions:
        session.execute_write(createPosition, position)  

# Creating Relations

### (Season) - [:HAS_GW]-> (Gameweek)

In [ ]:
#season : season_name 
#Gw : season , gw

# MATCH (keanu:Person {name:'Keanu Reeves'})
# MATCH (m:Movie {title:'Matrix Reloaded'})
# MERGE (keanu)-[:ACTED_IN]->(m);

# def createPosition(tx , name):
#     tx.run("CREATE (t:Position {name: $name})", name=name)

def SeasonHasGw(tx , season_name) :

    query = "MATCH (s:Season {season_name:$season_name})" \
    "MATCH (g:Gameweek {season:$season_name})" \
    "MERGE (s)-[:HAS_GW]->(g)"

    tx.run(query , season_name=season_name) 


In [ ]:
seasons = df['season'].unique()
seasons

array(['2021-22', '2022-23'], dtype=object)

In [ ]:
with driver.session() as session:
    for season in seasons:
        session.execute_write(SeasonHasGw, season)  

### 2. (Gameweek) - [:HAS_FIXTURE]-> (Fixture)

In [ ]:
# Gw : season , gw
# fixture : season , fixture , (kickoff)

def GwHasFixture(tx , season, GW_number, fixture_number) :

    query = "MATCH (g:Gameweek {season:$season , GW_number:$GW_number})" \
    "MATCH (f:Fixture {season:$season, fixture_number:$fixture_number})" \
    "MERGE (g)-[:HAS_FIXTURE]->(f)"

    tx.run(query , season=season, GW_number=GW_number, fixture_number=fixture_number) 



In [ ]:
has_fixtures = df[['season' ,'GW' , 'fixture']].drop_duplicates().sort_values(by='fixture')
has_fixtures

,season,GW,fixture
1109,2021-22,1,1
225,2022-23,1,1
766,2022-23,1,2
0,2021-22,1,2
415,2022-23,1,3
...,...,...,...
917,2021-22,38,378
376,2022-23,38,379
300,2021-22,38,379
537,2022-23,38,380


In [ ]:
with driver.session() as session:
    for _ , row in has_fixtures.iterrows():
        session.execute_write(GwHasFixture, row["season"],row['GW'], row["fixture"])


### 3.(Fixture) - [:HAS_HOME_TEAM]-> (Team)

In [ ]:
# fixture : season , fixture , (kickoff)
# team : name 

def GwHasHomeTeam(tx , season, fixture_number,  home_team) :

    query = "MATCH (f:Fixture {season:$season , fixture_number:$fixture_number})" \
    "MATCH (t:Team {name:$home_team})" \
    "MERGE (f)-[:HAS_HOME_TEAM]->(t)"

    tx.run(query , season=season , home_team=home_team , fixture_number=fixture_number) 


In [ ]:
home_team = df[['season' , 'fixture' , 'home_team']].drop_duplicates().sort_values(by='fixture')
home_team

,season,fixture,home_team
1109,2021-22,1,Brentford
225,2022-23,1,Crystal Palace
766,2022-23,2,Bournemouth
0,2021-22,2,Burnley
415,2022-23,3,Everton
...,...,...,...
917,2021-22,378,Liverpool
376,2022-23,379,Man Utd
300,2021-22,379,Man City
537,2022-23,380,Southampton


In [ ]:
with driver.session() as session:
    for _, row in home_team.iterrows():
        session.execute_write(GwHasHomeTeam, row["season"],row['fixture'], row["home_team"])

BufferError: Existing exports of data: object cannot be re-sized

### 4.(Fixture) - [:HAS_AWAY_TEAM]-> (Team)

In [ ]:
# fixture : season , fixture , (kickoff)
# team : name 

def GwHasAwayTeam(tx , season, fixture_number,  away_team) :

    query = "MATCH (f:Fixture {season:$season , fixture_number:$fixture_number})" \
    "MATCH (t:Team {name:$away_team})" \
    "MERGE (f)-[:HAS_AWAY_TEAM]->(t)"

    tx.run(query , season=season , away_team=away_team , fixture_number=fixture_number) 


In [ ]:
away_team = df[['season' , 'fixture' , 'away_team']].drop_duplicates().sort_values(by='fixture')
away_team

,season,fixture,away_team
1109,2021-22,1,Arsenal
225,2022-23,1,Arsenal
766,2022-23,2,Aston Villa
0,2021-22,2,Brighton
415,2022-23,3,Chelsea
...,...,...,...
917,2021-22,378,Wolves
376,2022-23,379,Fulham
300,2021-22,379,Aston Villa
537,2022-23,380,Liverpool


In [ ]:
with driver.session() as session:
    for _, row in away_team.iterrows():
        session.execute_write(GwHasAwayTeam, row["season"],row['fixture'], row["away_team"])

### 5.(Player) - [:PLAYS_AS]-> (Position)

In [ ]:
def PlayerPlaysAsPosition(tx , player_name , player_element , name ) :

    query = "MATCH (f:Player {player_name:$player_name , player_element:$player_element})" \
    "MATCH (p:Position {name:$name})" \
    "MERGE (f)-[:PLAYS_AS]->(p)"

    tx.run(query , player_name=player_name , player_element=player_element , name=name) 

In [ ]:
playerPositions = df[['name' ,'element' , 'position']].drop_duplicates()
playerPositions

,name,element,position
0,Aaron Connolly,72,FWD
38,Aaron Cresswell,411,DEF
76,Aaron Cresswell,457,DEF
114,Aaron Hickey,510,DEF
152,Aaron Lennon,575,MID
...,...,...,...
51796,Zidane Iqbal,551,MID
51834,Álvaro Fernández,556,GK
51866,Álvaro Fernández,728,DEF
51876,Çaglar Söyüncü,217,DEF


In [ ]:
with driver.session() as session:
    for _, row in playerPositions.iterrows():
        session.execute_write(PlayerPlaysAsPosition, row["name"],row['element'], row["position"])

BufferError: Existing exports of data: object cannot be re-sized

### 6.(Player) - [:PLAYED_IN]-> (Fixture)

In [ ]:
def PlayerPlayedInFixture(tx, player_name, player_element, season, fixture_number):
    query = "MATCH (p:Player {player_name:$player_name , player_element:$player_element})" \
    "MATCH (f:Fixture {season:$season , fixture_number:$fixture_number})" \
    "MERGE (p)-[:PLAYED_IN]->(f)"

    tx.run(query, player_name=player_name, player_element=player_element, season=season, fixture_number=fixture_number)

In [ ]:
playedInDF = df[df["minutes"] > 0][["name", "season", "fixture"]].drop_duplicates()
playedInDF


,name,season,fixture
1,Aaron Connolly,2021-22,12
5,Aaron Connolly,2021-22,54
14,Aaron Connolly,2021-22,146
15,Aaron Connolly,2021-22,163
38,Aaron Cresswell,2021-22,9
...,...,...,...
51944,Çaglar Söyüncü,2022-23,305
51945,Çaglar Söyüncü,2022-23,317
51946,Çaglar Söyüncü,2022-23,323
51947,Çaglar Söyüncü,2022-23,337


In [ ]:
with driver.session as session:
    for _, row in playedInDF.iterrows():
        session.execute_write(PlayerPlayedInFixture, row["name"], row["season"], row["fixture"])

In [ ]:
driver.close()